# Plotting APPL market data

Import libraries

In [1]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import plotly.offline as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots

yf.pdr_override()
py.init_notebook_mode(connected=True)
pd.options.plotting.backend = "plotly"

Get the dates we need for the project. We will use data from the past 5 years.

In [2]:
end_date = dt.datetime.now()
start_date = dt.datetime(2017, 1, 1)
start_date, end_date

(datetime.datetime(2017, 1, 1, 0, 0),
 datetime.datetime(2022, 12, 30, 16, 55, 10, 355574))

Create a stock list

In [3]:
stocklist = ['AAPL']

Create dataframe and fill it with stock data with pandas_datareader

In [4]:
df = pdr.get_data_yahoo(stocklist, start=start_date, end=end_date)
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,28.950001,29.082500,28.690001,29.037500,27.174751,115127600
2017-01-04,28.962500,29.127501,28.937500,29.004999,27.144342,84472400
2017-01-05,28.980000,29.215000,28.952499,29.152500,27.282375,88774400
2017-01-06,29.195000,29.540001,29.117500,29.477501,27.586529,127007600
2017-01-09,29.487499,29.857500,29.485001,29.747499,27.839207,134247600


Create two moving average columns. One for the 50 MA and one for the 200 MA

In [5]:
df['50MA']  = df['Close'].rolling(window=50,  min_periods=0).mean()
df['200MA'] = df['Close'].rolling(window=200, min_periods=0).mean()

df[['50MA','200MA']].head()

,50MA,200MA
Date,,
2017-01-03,29.037500,29.037500
2017-01-04,29.021250,29.021250
2017-01-05,29.065000,29.065000
2017-01-06,29.168125,29.168125
2017-01-09,29.284000,29.284000


Create the subplot figure

In [6]:
subplot_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=('AAPL', 'Volume'), row_width=[0.2, 0.7])

In [7]:
subplot_fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='AAPL'), row=1, col=1)

subplot_fig.add_trace(go.Scatter(x=df.index, y=df['50MA'],   marker_color='yellow', name='50MA'),   row=1, col=1)
subplot_fig.add_trace(go.Scatter(x=df.index, y=df['200MA'],  marker_color='blue',   name='200MA'),  row=1, col=1)
subplot_fig.add_trace(go.Bar(    x=df.index, y=df['Volume'], marker_color='orange', name='Volume'), row=2, col=1)

subplot_fig.update_layout(title = "Apple Inc. (AAPL) Stock Price History",
                          xaxis_tickfont_size=12,
                          yaxis=dict(title='AAPL Stock Price'),
                          autosize=False,
                          paper_bgcolor='rgba(0,0,0,0)')

subplot_fig.update(layout_xaxis_rangeslider_visible=False)